<a href="https://colab.research.google.com/github/NAVEED261/MY-AI-ASSISTANT/blob/main/thread_two_6_2_enhanced_chatbot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot with message summarization & external DB memory

## Review

We've covered how to customize graph state schema and reducer.

We've also shown a number of tricks for trimming or filtering messages in graph state.

We've used these concepts in a Chatbot with memory that produces a running summary of the conversation.

## Goals

But, what if we want our Chatbot to have memory that persists indefinitely?

Now, we'll introduce some more advanced checkpointers that support external databases.

Here, we'll show how to use [Postgres as a checkpointer](https://langchain-ai.github.io/langgraph/how-tos/persistence_postgres/)

In [12]:
%%capture --no-stderr
%pip install -U langgraph langgraph-checkpoint-postgres psycopg psycopg-pool langchain_google_genai


In [13]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

In [14]:
import os
os.environ["Langchain_api_key"] = userdata.get('Langchain_api_key')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

# Use sync connection¶
This sets up a synchronous connection to the database.

Synchronous connections execute operations in a blocking manner, meaning each operation waits for completion before moving to the next one. The DB_URI is the database connection URI, with the protocol used for connecting to a PostgreSQL database, authentication, and host where database is running. The connection_kwargs dictionary defines additional parameters for the database connection.

In [15]:
from google.colab import userdata
DB_URI = userdata.get('DB_URI')

In [16]:
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver

# Connection pool for efficient database access
connection_kwargs = {"autocommit": True, "prepare_threshold": 0}

# Create a persistent connection pool
pool = ConnectionPool(conninfo=DB_URI, max_size=20, kwargs=connection_kwargs)

# Initialize PostgresSaver checkpointer
checkpointer = PostgresSaver(pool)
checkpointer.setup()  # Ensure database tables are set up


Let's re-define our chatbot.

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

from langgraph.graph import END
from langgraph.graph import MessagesState

model: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key =  GEMINI_API_KEY)

class State(MessagesState):
    summary: str

# Define the logic to call the model
def call_model(state: State) -> State:

    # Get summary if it exists
    summary = state.get("summary", "")
    print(f"Using summary: {summary}")

    # If there is summary, then we add it
    if summary:

        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]

    else:
        messages = state["messages"]

    response = model.invoke(messages)
    return {"messages": response}

def summarize_conversation(state: State) -> State:
    print(f"Messages before summarizing: {len(state['messages'])}")
    # First, we get any existing summary
    summary = state.get("summary", "")
    print(f"Existing summary: {summary}")

    # Create our summarization prompt
    if summary:

        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )

    else:
        summary_message = "Create a summary of the conversation above:"


    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # Summarization logic
    print(f"New summary: {response.content}")

    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]

    print(f"Messages after truncation: {len(delete_messages)}")
    return {"summary": response.content, "messages": delete_messages}

# Determine whether to end or summarize the conversation
def should_continue(state: State) -> State:

    """Return the next node to execute."""

    messages = state["messages"]
    print(f"Message count: {len(messages)}")
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"

    # Otherwise we can just end
    return END

Now, we just re-compile with our postgres checkpointer.

In [20]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph

# Redefine workflow
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile the workflow with PostgreSQL checkpointer
graph = workflow.compile(checkpointer=checkpointer)


Now, we can invoke the graph several times.

In [21]:
# Configuration for thread
config = {"configurable": {"thread_id": "2"}}

# Start a conversation
input_message = HumanMessage(content="assalamualykum , how r u? ")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 


Message count: 2
================================== Ai Message ==================================

Wa alaikum assalam!  I am doing well, thank you for asking. How are you?


StateSnapshot(values={'messages': [HumanMessage(content='assalamualykum , how r u? ', additional_kwargs={}, response_metadata={}, id='c1ee5f66-bad2-41dc-90b4-359d17b95970'), AIMessage(content='Wa alaikum assalam!  I am doing well, thank you for asking. How are you?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a67ba83c-87e1-4f32-b23b-92f96b102560-0', usage_metadata={'input_tokens': 12, 'output_tokens': 22, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}})]}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efc8f7e-9969-6cf2-8001-788084455284'}}, metadata={'step': 1, 'source': 'loop', 'writes': {'conversation': {'messages': AIMessage(content='Wa alaikum assalam!  I am doing well, thank you for asking. How are you?\n', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'safety_ratings': [], 'prompt_feed

In [22]:
# Configuration for thread
config = {"configurable": {"thread_id": "2"}}

# Start a conversation
input_message = HumanMessage(content="im learning generative ai")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 


Message count: 4
================================== Ai Message ==================================

That's fantastic! Generative AI is a fascinating and rapidly evolving field.  What aspects of generative AI are you focusing on right now?  Are you working with specific models, datasets, or applications?  I'd be happy to hear more about your learning journey and perhaps offer some suggestions or resources if you'd like.


StateSnapshot(values={'messages': [HumanMessage(content='assalamualykum , how r u? ', additional_kwargs={}, response_metadata={}, id='c1ee5f66-bad2-41dc-90b4-359d17b95970'), AIMessage(content='Wa alaikum assalam!  I am doing well, thank you for asking. How are you?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a67ba83c-87e1-4f32-b23b-92f96b102560-0', usage_metadata={'input_tokens': 12, 'output_tokens': 22, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='im learning generative ai', additional_kwargs={}, response_metadata={}, id='dd16f71d-5001-4f80-be51-eaaba66bc9e9'), AIMessage(content="That's fantastic! Generative AI is a fascinating and rapidly evolving field.  What aspects of generative AI are you focusing on right now?  Are you working with specific models, datasets, or applications?  I'd be happy to hear more about your learning 

In [23]:
# Configuration for thread
config = {"configurable": {"thread_id": "2"}}

# Start a conversation
input_message = HumanMessage(content="what is im learning?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 


Message count: 6
================================== Ai Message ==================================

You said you are learning **Generative AI**.  That's a broad field encompassing many techniques and applications.  To better understand what *specific* aspects you're learning, I need more information from you.  For example, are you learning about:

* **Specific models:**  Like GANs (Generative Adversarial Networks), VAEs (Variational Autoencoders), Transformers, diffusion models, etc.?
* **Programming languages:**  Python is commonly used, often with libraries like TensorFlow or PyTorch.
* **Applications:**  Image generation, text generation, music generation, code generation, etc.?
* **Theoretical concepts:**  Like probability distributions, loss functions, backpropagation, etc.?

Knowing more about your learning path will help me understand what you're working on and how I can assist you.


StateSnapshot(values={'messages': [HumanMessage(content='assalamualykum , how r u? ', additional_kwargs={}, response_metadata={}, id='c1ee5f66-bad2-41dc-90b4-359d17b95970'), AIMessage(content='Wa alaikum assalam!  I am doing well, thank you for asking. How are you?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a67ba83c-87e1-4f32-b23b-92f96b102560-0', usage_metadata={'input_tokens': 12, 'output_tokens': 22, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='im learning generative ai', additional_kwargs={}, response_metadata={}, id='dd16f71d-5001-4f80-be51-eaaba66bc9e9'), AIMessage(content="That's fantastic! Generative AI is a fascinating and rapidly evolving field.  What aspects of generative AI are you focusing on right now?  Are you working with specific models, datasets, or applications?  I'd be happy to hear more about your learning 

In [24]:
# Configuration for thread
config = {"configurable": {"thread_id": "2"}}

# Start a conversation
input_message = HumanMessage(content="do u know about Generative AI?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 


Message count: 8
Messages before summarizing: 8
Existing summary: 


New summary: The conversation began with a greeting in Arabic.  I then learned that the user is studying Generative AI.  We then engaged in a discussion clarifying what aspects of Generative AI the user is focusing on, with me explaining that the field is broad and encompasses various models (GANs, VAEs, Transformers, etc.), programming languages (like Python), and applications (image, text, music generation, etc.).  Finally, I confirmed my own extensive knowledge of Generative AI, including its models, underlying mathematics, applications, and ethical considerations.  The conversation concluded with an offer to assist the user further in their learning.

Messages after truncation: 6
================================== Ai Message ==================================

Yes, I know about Generative AI.  I have been trained on a massive dataset of text and code, which includes a significant amount of information about generative AI models, techniques, and applications.  My knowledge encompass

StateSnapshot(values={'messages': [HumanMessage(content='do u know about Generative AI?', additional_kwargs={}, response_metadata={}, id='cc2cf9f8-1668-4eac-ae89-fe341f8a56d6'), AIMessage(content="Yes, I know about Generative AI.  I have been trained on a massive dataset of text and code, which includes a significant amount of information about generative AI models, techniques, and applications.  My knowledge encompasses:\n\n* **Various generative models:**  GANs, VAEs, diffusion models (like Stable Diffusion), transformers (like GPT models), autoregressive models, and more.  I understand their architectures, strengths, weaknesses, and training processes.\n\n* **Underlying mathematical concepts:**  I understand the probability distributions, optimization algorithms (like gradient descent), and loss functions used in training these models.\n\n* **Applications across different domains:**  I'm aware of the use of generative AI in image generation, text generation (like writing stories, su

In [25]:
# Retrieve state using thread ID
config = {"configurable": {"thread_id": "2"}}
graph_state = graph.get_state(config)
graph_state

StateSnapshot(values={'messages': [HumanMessage(content='do u know about Generative AI?', additional_kwargs={}, response_metadata={}, id='cc2cf9f8-1668-4eac-ae89-fe341f8a56d6'), AIMessage(content="Yes, I know about Generative AI.  I have been trained on a massive dataset of text and code, which includes a significant amount of information about generative AI models, techniques, and applications.  My knowledge encompasses:\n\n* **Various generative models:**  GANs, VAEs, diffusion models (like Stable Diffusion), transformers (like GPT models), autoregressive models, and more.  I understand their architectures, strengths, weaknesses, and training processes.\n\n* **Underlying mathematical concepts:**  I understand the probability distributions, optimization algorithms (like gradient descent), and loss functions used in training these models.\n\n* **Applications across different domains:**  I'm aware of the use of generative AI in image generation, text generation (like writing stories, su

In [26]:
config = {"configurable": {"thread_id": "2"}}
graph_state = graph.get_state(config).values.get("messages")
graph_state

[HumanMessage(content='do u know about Generative AI?', additional_kwargs={}, response_metadata={}, id='cc2cf9f8-1668-4eac-ae89-fe341f8a56d6'),
 AIMessage(content="Yes, I know about Generative AI.  I have been trained on a massive dataset of text and code, which includes a significant amount of information about generative AI models, techniques, and applications.  My knowledge encompasses:\n\n* **Various generative models:**  GANs, VAEs, diffusion models (like Stable Diffusion), transformers (like GPT models), autoregressive models, and more.  I understand their architectures, strengths, weaknesses, and training processes.\n\n* **Underlying mathematical concepts:**  I understand the probability distributions, optimization algorithms (like gradient descent), and loss functions used in training these models.\n\n* **Applications across different domains:**  I'm aware of the use of generative AI in image generation, text generation (like writing stories, summarizing text, translating langu

In [ ]:
pool.close()



### Persisting state

Using database like Postgres means state is persisted!

For example, we can re-start the notebook kernel and see that we can still load from Postgres DB on disk.
